# Hey, maikol here

In [1]:
# For testing and showing
%load_ext autoreload
%autoreload 2

### File functions

- `make_dirs`

In [2]:
from maikol_utils.file_utils import make_dirs

dirs = ["./data/input", "./data/output"]
# Safely creates all these directories if they don't exist
# Return whether the directories EXISTED created or not
# -> True, no need to create them, it was already there
# -> False, they were created now
dirs_existed = make_dirs(dirs)
dirs_existed

[False, False]

- `check_dirs_existance`

In [3]:
from maikol_utils.file_utils import check_dirs_existance
from maikol_utils.print_utils import print_separator

dirs = ["./data", "./config", "./logs", "./outputs"]

# ==========================================================================================
print_separator("Check dirs existance with raise error")
# Raises KeyError if any path is missing
try:
    check_dirs_existance(dirs)
except KeyError as e:
    print(e)  # e.g. "Some paths were not found: ['./config', './logs']"


# ==========================================================================================
print_separator("Check dirs existance without raise error")
missing_dirs = check_dirs_existance(dirs, raise_error=False)
print(missing_dirs)  


________________________________________________________________
             Check dirs existance with raise error              

"Some paths were not found: ['./config', './outputs']"
________________________________________________________________
            Check dirs existance without raise error            

['./config', './outputs']


- `save_json` & `load_json`

In [4]:
from maikol_utils.file_utils import save_json, load_json

data = {"users": ["alice", "bob"], "count": 2}
# Saves to outputs/users.json, creating the folder if needed
save_json("outputs/users.json", data)

# Attempt to load; if not found, returns {}
settings = load_json("outputs/users.json")
print(settings)  # {"users": ["alice", "bob"], "count": 2}

Saving output at outputs/users.json...
Loading output from outputs/users.json...
{'users': ['alice', 'bob'], 'count': 2}


- `clear_directories`

In [5]:
from maikol_utils.file_utils import clear_directories
from maikol_utils.print_utils import print_separator

dirs = ["./data", "./config", "./logs", "./outputs"]
print_separator()
clear_directories(dirs[0], remove_folder=False)

print_separator()
clear_directories(dirs, remove_folder=False)

print_separator()
clear_directories(dirs, remove_folder=True)



________________________________________________________________
Cleared ./data
________________________________________________________________
Cleared ./data
⚠️'./config' not found or not a dir⚠️
Cleared ./logs
Cleared ./outputs
________________________________________________________________
Removed ./data
⚠️'./config' not found or not a dir⚠️
Removed ./logs
Removed ./outputs


- `list_dir_files`

In [6]:
from maikol_utils.file_utils import list_dir_files
from maikol_utils.print_utils import print_separator

print_separator("Basic call", sep_type="LONG")
dir_list, n_files = list_dir_files("./tests")
print(f"{n_files = }")
print(f"{dir_list = }")

print_separator("Max number of files", sep_type="LONG")
dir_list, n_files = list_dir_files("./tests", max_files=5)
print(f"{n_files = }")
print(f"{dir_list = }")

print_separator("Non existing path -> Error", sep_type="LONG")
print(list_dir_files("./AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"))

print_separator("Natural sorting and python sorting", sep_type="LONG")
print(list_dir_files("./tests")[0])
print(list_dir_files("./tests", nat_sorting=False)[0])

print_separator("Recursivelly get files in subfolders", sep_type="LONG")
print(list_dir_files("./tests", recursive=True, max_files=7)[0])

print_separator("Remove the absolute path from the returned files", sep_type="LONG")
print(list_dir_files("./tests", recursive=True, max_files=7, absolute_path=False)[0])

________________________________________________________________________________________________________________________________
                                                           Basic call                                                           

n_files = 6
dir_list = ['./tests/test.py', './tests/x_1.txt', './tests/x_2.txt', './tests/x_10.txt', './tests/x_20.txt', './tests/x_21.txt']
________________________________________________________________________________________________________________________________
                                                      Max number of files                                                       

n_files = 5
dir_list = ['./tests/test.py', './tests/x_1.txt', './tests/x_2.txt', './tests/x_10.txt', './tests/x_20.txt']
________________________________________________________________________________________________________________________________
                                                   Non existing path -> Error              

Look at the 'Natural sorting and python sorting'. The order is 'natural', from prioritizing 1 over 10, 10 over 2 -> 1 over 2, and 2 over 10.

- manage_temp_files

In [7]:
import time
from typing import Any
from maikol_utils.file_utils import manage_temp_files, clear_directories
from maikol_utils.print_utils import print_separator, clear_status

def long_task_llm(ia_model: Any, page: str, extra: int = None):
    time.sleep(1)
    res = page.split()
    return res, f"ia_model: {ia_model}, res: {res}{f'-{extra}' if extra is not None else ''}"

ia_model = "IA"
pages = [
    f"{i}? "*i for i in range(10)
]

print_separator("Simulating process starts")
for i in range(3):
    manage_temp_files(
        i,                     # call_key
        "./temp",              # temp_folder_path
        long_task_llm,         # function
        True,                  # manage_temp_files _verbose
        True,                  # keep_logs
        ia_model,              # *args -> function arg 1
        pages[i],              # *args -> function arg 2
        extra=10,              # **kwargs -> function keyword arg
    )

clear_status()
print_separator("Simulating process stop")

for i in range(6):
    out1, out2 = manage_temp_files(
        i,                     # call_key
        "./temp",              # temp_folder_path
        long_task_llm,         # function
        True,                  # manage_temp_files _verbose
        True,                  # keep_logs
        ia_model,              # *args -> function arg 1
        pages[i],              # *args -> function arg 2
        extra=10,              # **kwargs -> function keyword arg
    )

clear_status()
print_separator("DONE!")
clear_directories(["temp/"], remove_folder=True)


________________________________________________________________
                   Simulating process starts                    

 - Executing 'long_task_llm' for '0'...
 - Executing 'long_task_llm' for '1'...
 - Executing 'long_task_llm' for '2'...
________________________________________________________________                                                        
                    Simulating process stop                     

 - Skiping '0': Temp file found at './temp/0.json'
 - Skiping '1': Temp file found at './temp/1.json'
 - Skiping '2': Temp file found at './temp/2.json'
 - Executing 'long_task_llm' for '3'...
 - Executing 'long_task_llm' for '4'...
 - Executing 'long_task_llm' for '5'...
________________________________________________________________                                                        
                             DONE!                              

Removed temp/


### Printing

- `print_separator`

In [8]:
from maikol_utils.print_utils import print_separator

print_separator()
print_separator(" ") # creates a separation line after the live

print_separator("START PROCESS", sep_type="START")
print_separator("FINAL RESULTS", sep_type="SUPER")
print_separator("STEP COMPLETE", sep_type="SHORT")


print_separator(" RED LINE", color="red")
print_separator(" GREEN LINE", color="green")
print_separator(" BLUE LINE", color="blue")
print_separator(" YELLOW LINE", color="yellow")


________________________________________________________________
________________________________________________________________
                                                                


                                                         START PROCESS                                                          


                                                         FINAL RESULTS                                                          

________________________________
         STEP COMPLETE          

________________________________________________________________
                            RED LINE                            

________________________________________________________________
                           GREEN LINE                           

________________________________________________________________
                            BLUE LINE                           

________________________________________________________________
                 

- `print_color` & `print_warn`

In [9]:
from maikol_utils.print_utils import print_color, print_warn, print_error, _colors, print_separator

print_color("All systems go", color="green")
print_color("Unexpected value", color="red")

# Prints ⚠️Missing config file!⚠️ in yellow
print_warn("Missing config file!")

# Prints ❌Missing config file!❌ in yellow
text = print_error("No model found!")

print_separator(" ") 
for color in _colors.keys():
    print_color(f"This is {color} color", color=color)

All systems go
Unexpected value
⚠️Missing config file!⚠️
❌No model found!❌
________________________________________________________________
                                                                

This is black color
This is red color
This is green color
This is yellow color
This is blue color
This is purple color
This is cyan color
This is white color
This is reset color
This is italic color
This is bold_italic color
This is underline color
This is strikethrough color
This is bold_black color
This is bold_red color
This is bold_green color
This is bold_yellow color
This is bold_blue color
This is bold_purple color
This is bold_cyan color
This is bold_white color
This is underline_black color
This is underline_red color
This is underline_green color
This is underline_yellow color
This is underline_blue color
This is underline_purple color
This is underline_cyan color
This is underline_white color
This is bg_black color
This is bg_red color
This is bg_green color
This is bg_yel

- `print_status`
    - this wont work here :)
    - Look at the '>>>' examples

In [10]:
import time
from maikol_utils.print_utils import print_status, clear_status

print("Processing document...")


n_files = 3
for i in range(1, n_files+1):
    print_status(f" - Processing page {i}/{n_files}...")
    time.sleep(0.5)
# At the end call clear_status() to clear the previous line
clear_status()

print("DONE!")


"""
>>> print("Processing document...")
>>> Processing document...

>>> print_status(f" - Processing page {i}/{n_files}...")
>>> Processing document...
>>>  - Processing page 1/3

>>> print_status(f" - Processing page {i}/{n_files}...")
>>> Processing document...
>>>  - Processing page 2/3

>>> print_status(f" - Processing page {i}/{n_files}...")
>>> Processing document...
>>>  - Processing page 3/3

>>> clear_status()
>>> print("DONE!")
>>> Processing document...
>>> DONE!
"""

Processing document...
DONE!ocessing page 3/3...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       


'\n>>> print("Processing document...")\n>>> Processing document...\n\n>>> print_status(f" - Processing page {i}/{n_files}...")\n>>> Processing document...\n>>>  - Processing page 1/3\n\n>>> print_status(f" - Processing page {i}/{n_files}...")\n>>> Processing document...\n>>>  - Processing page 2/3\n\n>>> print_status(f" - Processing page {i}/{n_files}...")\n>>> Processing document...\n>>>  - Processing page 3/3\n\n>>> clear_status()\n>>> print("DONE!")\n>>> Processing document...\n>>> DONE!\n'

- `clear_bash` & `print_clear_bash`
    - this wont work here :)
    - Look at the '>>>' examples

In [11]:
import time
from maikol_utils.print_utils import clear_bash, print_clear_bash


print("Line A")
time.sleep(1)
print("Line B")
time.sleep(1)
clear_bash(2)   # removes both lines from the terminal
print("Clean slate!")


print("Loading data...")
time.sleep(1)
print_clear_bash("✔️ Data loaded successfully!", n_lines=1)

"""

>>> print("Line A")
>>> Line A

>>> print("Line B")
>>> Line A
>>> Line B

>>> clear_bash(2) 
>>> 

>>> print("Clean slate!")
>>> Clean state!

>>> print("Loading data...")
>>> Clean state!
>>> Loading data...

>>> print_clear_bash("✔️ Data loaded successfully!", n_lines=1)
>>> Clean state!
>>> ✔️ Data loaded successfully!
"""


Line A
Line B
Clean slate!
Loading data...
✔️ Data loaded successfully!


'\n\n>>> print("Line A")\n>>> Line A\n\n>>> print("Line B")\n>>> Line A\n>>> Line B\n\n>>> clear_bash(2) \n>>> \n\n>>> print("Clean slate!")\n>>> Clean state!\n\n>>> print("Loading data...")\n>>> Clean state!\n>>> Loading data...\n\n>>> print_clear_bash("✔️ Data loaded successfully!", n_lines=1)\n>>> Clean state!\n>>> ✔️ Data loaded successfully!\n'

- `print_utf_8`

In [12]:
from maikol_utils.print_utils import print_utf_8, print_separator


badly_formed_text = "L\\u00f3pez\\nL\\u00ednea 2"

print_separator("No format")
print(badly_formed_text)

print_separator("Format")

print_utf_8(badly_formed_text)

print_separator("")

________________________________________________________________
                           No format                            

L\u00f3pez\nL\u00ednea 2
________________________________________________________________
                             Format                             

López
Línea 2
________________________________________________________________


### With logger

- Configuration

In [13]:
from dataclasses import dataclass
# ======================================================================================================               
#                                              CONFIGURATION
# ======================================================================================================
@dataclass
class Configuration:
    logs_folder:           str = r"logs"
    logs_path_file: str = fr"{logs_folder}/app.log"

CONFIG = Configuration()

- Loger

In [14]:
import os
import logging


from logging.handlers import TimedRotatingFileHandler

# ======================================================================================================               
#                                              LOGGER FOR PRINT
# ======================================================================================================
class ColorFormatter(logging.Formatter):
    COLORS = {
        logging.DEBUG:    "\033[32m",  # green
        logging.INFO:     "\033[34m",  # blue
        logging.WARNING:  "\033[33m",  # yellow-orange
        logging.ERROR:    "\033[31m",  # red
        logging.CRITICAL: "\033[1;31m" # bright red
    }
    RESET = "\033[0m"

    def format(self, record):
        color = self.COLORS.get(record.levelno, "")
        record.levelname = f"{color}{record.levelname}{self.RESET}"
        return super().format(record)

os.makedirs(CONFIG.logs_folder, exist_ok=True)
file_handler = TimedRotatingFileHandler(
    CONFIG.logs_path_file, when="midnight", interval=1, backupCount=7, encoding="utf-8"
)
file_handler.setFormatter(logging.Formatter(
    "%(asctime)s | %(levelname)s: %(message)s", "%Y-%m-%d %H:%M"
))

# Create shared handler with your formatter
handler = logging.StreamHandler()
handler.setFormatter(ColorFormatter(
    "%(asctime)s | %(levelname)s: %(message)s", "%Y-%m-%d %H:%M"
))

# Apply to your logger
logger = logging.getLogger(__name__)
logger.handlers.clear()
logger.propagate = False
logger.setLevel(logging.INFO)
logger.addHandler(handler)
logger.addHandler(file_handler)

# Also apply to FastAPI and Uvicorn loggers
for name in ["uvicorn", "uvicorn.access", "uvicorn.error"]:
    log = logging.getLogger(name)
    log.handlers.clear()
    log.propagate = False
    log.setLevel(logging.INFO)
    log.addHandler(handler)

- Example

In [15]:
from maikol_utils.config import set_logger
set_logger(logger)  


In [16]:
import time
from maikol_utils.print_utils import clear_bash, print_clear_bash, print_log

print_log("Line A")
time.sleep(1)
print_log("Line B")
time.sleep(1)
clear_bash(2)   # removes both lines from the terminal
print_log("Clean slate!")


print_log("Loading data...")
time.sleep(1)
print_clear_bash("✔️ Data loaded successfully!", n_lines=1)

"""

>>> print("Line A")
>>> 2025-07-21 09:53 | INFO: Line A

>>> print("Line B")
>>> 2025-07-21 09:53 | INFO: Line A
>>> 2025-07-21 09:53 | INFO: Line B

>>> clear_bash(2) 
>>> 

>>> print("Clean slate!")
>>> 2025-07-21 09:53 | INFO: Clean state!

>>> print("Loading data...")
>>> 2025-07-21 09:53 | INFO: Clean state!
>>> 2025-07-21 09:53 | INFO: Loading data...

>>> print_clear_bash("✔️ Data loaded successfully!", n_lines=1)
>>> 2025-07-21 09:53 | INFO: Clean state!
>>> 2025-07-21 09:53 | INFO: ✔️ Data loaded successfully!
"""


2025-10-08 09:40 | INFO: Line A
2025-10-08 09:40 | INFO: Line B
2025-10-08 09:40 | INFO: 
2025-10-08 09:40 | INFO: Clean slate!
2025-10-08 09:40 | INFO: Loading data...
2025-10-08 09:40 | INFO: 
2025-10-08 09:40 | INFO: ✔️ Data loaded successfully!


'\n\n>>> print("Line A")\n>>> 2025-07-21 09:53 | INFO: Line A\n\n>>> print("Line B")\n>>> 2025-07-21 09:53 | INFO: Line A\n>>> 2025-07-21 09:53 | INFO: Line B\n\n>>> clear_bash(2) \n>>> \n\n>>> print("Clean slate!")\n>>> 2025-07-21 09:53 | INFO: Clean state!\n\n>>> print("Loading data...")\n>>> 2025-07-21 09:53 | INFO: Clean state!\n>>> 2025-07-21 09:53 | INFO: Loading data...\n\n>>> print_clear_bash("✔️ Data loaded successfully!", n_lines=1)\n>>> 2025-07-21 09:53 | INFO: Clean state!\n>>> 2025-07-21 09:53 | INFO: ✔️ Data loaded successfully!\n'

- reset logger

In [17]:
from maikol_utils.config import set_logger
set_logger(None)  

# Other utils

- Instantiate dataclass from arguments

In [19]:
import os
import argparse
from dataclasses import dataclass
from maikol_utils.other_utils import args_to_dataclass
from maikol_utils.file_utils import make_dirs

@dataclass
class Configuration:
    logs_folder: str = "logs"
    logs_path_file: str = "logs/app.log"
    model_name: str = "default_model"
    learning_rate: float = 0.001

    def __post_init__(self):
        # Ensure logs folder exists
        self.logs_path_file = os.path.join(self.logs_folder, "app.log")
        make_dirs(self.logs_folder)

```python
# =================================================
#                   EXAMPLE USAGE
# =================================================
if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    # Optional arguments (only these provided will override defaults)
    parser.add_argument("--logs_folder", type=str)
    parser.add_argument("--model_name", type=str)
    parser.add_argument("--learning_rate", type=float)
    args = parser.parse_args()

    # Convert parsed args → dataclass
    CONFIG = args_to_dataclass(args, Configuration) # <- important line ==============================

    print(CONFIG)
```

- From seconds to pretty print

In [ ]:
from maikol_utils.other_utils import parse_seconds_to_minutes
from maikol_utils.print_utils import print_log, print_color, print_separator


print_log(parse_seconds_to_minutes(3661))  # "1h 1m 1s"
print_log(parse_seconds_to_minutes(10))  # "1h 1m 1s"
print_log(parse_seconds_to_minutes(61)) # "1h 1m 1s"
print_log(parse_seconds_to_minutes(123094.2124)) # "1h 1m 1s"


print_separator(" ")

print_color(parse_seconds_to_minutes(90124.5124), color="green") 
print_color(parse_seconds_to_minutes(123.26324), color="bg_hi_blue") 
print_color(parse_seconds_to_minutes(5112.2623643), color="bold_hi_yellow") 


01 hrs, 01 mins, 01.0000 sec
10.0000 sec
01 mins, 01.0000 sec
34 hrs, 11 mins, 34.2124 sec
________________________________________________________________
                                                                

25 hrs, 02 mins, 04.5124 sec
02 mins, 03.2632 sec
01 hrs, 25 mins, 12.2623 sec


'\x1b01 hrs, 25 mins, 12.2623 sec\x1b'